In [ ]:
!git clone https://personalaccesstoken@github.com/username/reponame.git

Cloning into 'reponame'...
fatal: could not read Password for 'https://personalaccesstoken@github.com': No such device or address


In [1]:
!git clone https://github.com/jairotunior/dashboard_tutorial.git

Cloning into 'dashboard_tutorial'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 136 (delta 37), reused 127 (delta 28), pack-reused 0
Receiving objects: 100% (136/136), 683.72 KiB | 8.76 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
!ls

dashboard_tutorial  sample_data


In [3]:
import os
os.chdir('dashboard_tutorial/')

In [4]:
!ls

dashboards	    datasets   requirements.txt  screenshots
dashboard_tutorial  README.md  run_dashboard.py


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!ls

dashboards	    datasets   requirements.txt  screenshots
dashboard_tutorial  README.md  run_dashboard.py


In [11]:
!python3 run_dashboard.py

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Traceback (most recent call last):
  File "/content/dashboard_tutorial/run_dashboard.py", line 41, in <module>
    fred_source = FREDSource(fred_credentials=fred_credentials)
  File "/content/dashboard_tutorial/dashboard_tutorial/sources/fred_source.py", line 12, in __init__
    assert os.path.exists(fred_credentials), "La ruta suministrada para las credenciales no existe"
AssertionError: La ruta suministrada para las credenciales no existe


In [12]:
import os
import logging
from pathlib import Path
from dashboard_tutorial.sources import FREDSource, QuandlSource, FileSource
from dashboard_tutorial.transformers import FractionalDifferentiationEW, FractionalDifferentiationFFD, Differentiation, PercentageChange
from dashboard_tutorial.managers import ManagerDashboard
from dashboard_tutorial.views import DashboardView

if __name__ == '__main__':
    # Config Logging
    logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)

    BASE_DIR = Path(__file__).resolve().parent

    ANALYSIS_PATH = os.path.join(BASE_DIR, 'dashboards')

    if not os.path.exists(ANALYSIS_PATH):
        logging.info("[+] Se ha creado directorio de archivos.")
        os.mkdir(ANALYSIS_PATH)

    # Define Manager
    manager = ManagerDashboard(path=ANALYSIS_PATH)

    # **************** Register Transformers ***************************
    differentiation_transform = Differentiation(units_show='Returns')
    fractional_diff_transform = FractionalDifferentiationEW(units_show='Fractional Return')
    fractional_diff_ffd_transform = FractionalDifferentiationFFD(units_show='FFD Return')
    percentage_change_transform = PercentageChange(units_show='Percentage Change')
    percentage_change_from_year_transform = PercentageChange(name="Percentage Change from Year Ago", units_show='Percentage Change from Year Ago', periods=12)

    manager.transformers.register(fractional_diff_transform)
    manager.transformers.register(fractional_diff_ffd_transform)
    manager.transformers.register(differentiation_transform)
    manager.transformers.register(percentage_change_transform)
    manager.transformers.register(percentage_change_from_year_transform)


    # ******************* Create Source Managers ******************************
    fred_credentials = os.path.join(BASE_DIR, "api_fred.txt")
    fred_source = FREDSource(fred_credentials=fred_credentials)

    quandl_api_key = 'API_KEY'
    quandl_source = QuandlSource(api_key=quandl_api_key)

    # File source
    file_source_dir = os.path.join(BASE_DIR, "datasets", 'yields')
    file_source = FileSource(dir=file_source_dir)

    manager.sources.register(fred_source)
    manager.sources.register(quandl_source)
    manager.sources.register(file_source)

    manager.load()

    # ******************************************** Create Figures *************************************
    logging.info("[+] Render dashboards...")

    dashboard_view = DashboardView(title="Dashboard", manager=manager)

    dashboard_view.show()



ModuleNotFoundError: ignored

# Version 2

In [13]:
!pip install jupyter-dash==0.3.0
!pip install dash==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 950.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.21.6 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: dash
    Found existing installation: dash 2.13.0
    Uninstall

In [23]:
try:
   from werkzeug.debug.tbtools import DebugTraceback
except ImportError:  # temporary fallback for werkzeug<2.1
   from werkzeug.debug.tbtools import get_current_traceback as DebugTraceback

In [24]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
# app.run_server(mode='inline')
app.run_server(mode='inline',host="0.0.0.0",port=1005)

ImportError: ignored